## 4. [Execution model](https://docs.python.org/2/reference/executionmodel.html)

### 4.1 Naming and binding

- **Names** refer to objects. 

    - Names are introduced by name **binding** operations. 
    
    - Each occurrence of a name in the program text refers to the **binding** of that name established in the innermost function **block** containing the use.
    
    
- A **block** is a piece of Python program text that is executed as a unit. The following are blocks: 

    - a module, 
    - a function body, and 
    - a class definition. 
    - Each command typed interactively
    - A script file 
    - A script command (a command specified on the interpreter command line with the **‘-c‘** option) 
    - The file read by the built-in function **execfile() **
    - The string argument passed to the built-in function **eval()** and to the **exec** statement
    - The expression read and evaluated by the built-in function **input()** is a code block.
    
    
- A code block is executed in an **execution frame**. 

    - A frame contains some administrative information (used for debugging) and 
    - determines where and how execution continues after the code block’s execution has completed.
    
    
- A **scope** defines the visibility of a name within a block. 

    - A **local variable** defined in a block:
        - its scope includes that block. 
        - If the definition occurs in a **function block**, the scope extends to any blocks contained within the defining one, 
        - unless a contained block introduces a different binding for the name. 
    - The scope of names defined in a **class block** is limited to the class block; 
        - it does not extend to the code blocks of methods 
            – this includes generator expressions since they are implemented using a function scope. 
        - This means that the following will fail:

In [2]:
class A:
    a = 42
    b = list(a + i for i in range(10)) # NameError: global name 'a' is not defined

NameError: global name 'a' is not defined

- When a name is used in a code block, it is resolved using the **nearest** enclosing scope. 
- The set of all such scopes visible to a code block is called the block’s **environment**.

    - If a name is bound in a block, it is a **local** variable of that block. 

    - If a name is bound at the module level, it is a **global** variable. 
        - (The variables of the module code block are *local and global*.) 

    - If a variable is used in a code block but not defined there, it is a **free** variable.

In [17]:
x = 0
def foo():
    print(x)            # x is a global variable
    print(locals())
foo()

print

def bar():
    x = 1               # x is a bound variable
    def foo():
        print(x)        # x is a free variable
        print(locals())
    foo()
bar()

print

x = 42
def foo():
    global x
    def baz():
        print(x)        # x is a FREE GLOBAL variable
        print(locals())
    baz()
foo()

"""

http://stackoverflow.com/questions/12919278/how-to-define-free-variable-in-python

From what I have understood the documentation is indeed a bit ambiguous on free variables. 
There are free global variables which are treated as plain globals and lexically bound free variables.
Eli Bendersky sum's it up nicely in a blog post on symbol tables:


    Unfortunately, there's a shorthand in the core of Python that may initially confuse readers as to exactly 
    what constitutes a "free" variable. Fortunately, it's a very slight confusion that's easy to put in order. 
    The execution model reference says:

        If a variable is used in a code block but not defined there, it is a free variable.
        
    This is consistent with the formal definition. In the source, however, "free" is actually used as a shorthand for 
    "lexically bound free variable" (i.e. variables for which a binding has been found in an enclosing scope), with 
    "global" being used to refer to all remaining free variables. So when reading the CPython source code it is 
    important to remember that the full set of free variables includes both 
    
        the variables tagged specifically as "free", as well as 
        those tagged as "global".

    Thus, to avoid a confusion I say "lexically bound" when I want to refer to the variables actually treated in 
    CPython as free.


The reason why this shorthand was used is probably because when you have a global free variable there's really no change 
whatsoever in the bytecode emitted. If a global variable is 'free' or if it isn't doesn't change the fact that the 
look-up for that name will use LOAD_GLOBAL in both cases. So global free variables aren't all that special.

On the other hand, lexically bound variables are treated specially and are enclosed in cell objects, the objects are 
the storage space for lexically bound free variables and are located in the __closure__ attribute for a given function. 
A special LOAD_DEREF instruction is created for these that examines the cells present for the free variables. 
The description for the LOAD_DEREF instruction is:


    LOAD_DEREF(i)
    Loads the cell contained in slot i of the cell and free variable storage


So in Python free variables only make a difference as a concept in situations where a definition for an object that 
has state is lexically (i.e statically) nested in another definition for an object that has state.

"""

print

0
{}

1
{'x': 1}

42
{}



- When a name is **not** found at all, a **NameError** exception is raised. 
    - If the name refers to a local variable that has not been bound, a **UnboundLocalError** exception is raised. 
    - UnboundLocalError is a subclass of NameError.
    
    
- The following constructs **bind names**: 
    - formal parameters to functions, 
    - import statements, 
    - class and function definitions (these bind the class or function name in the defining block), and 
    - targets that are identifiers if occurring 
        - in an assignment, for loop header, 
        - in the second position of an except clause header or 
        - after as in a with statement. 
    - The import statement of the form **from ... import * ** binds **all** names defined in the imported module, 
        - except those beginning with an underscore:
            - This form may only be used at the module level.
            
- A target occurring in a del statement is also considered bound for this purpose 
    - (though the actual semantics are to unbind the name). 
- It is illegal to unbind a name that is referenced by an enclosing scope; the compiler will report a **SyntaxError**.

In [1]:
try:
    print aa
except Exception as e:
    print type(e), e # <type 'exceptions.NameError'> name 'aa' is not defined

print

def func0():
    bb = 'bb'
    print aa
try:
    func0()
except Exception as e:
    print type(e), e # <type 'exceptions.NameError'> global name 'aa' is not defined

print 

cc = 1
def func0():
    bb = 'bb'
    print cc
    cc = 2
try:
    func0()
except Exception as e:
    print type(e), e # <type 'exceptions.UnboundLocalError'> local variable 'cc' referenced before assignment

def func1():
    a = 1
    print 'a in func1():', a
    def nested_func():
        print 'a referenced in nested_func():', a
    del a # SyntaxError: can not delete variable 'a' referenced in nested scope
    
func1() 

<type 'exceptions.NameError'> name 'aa' is not defined

<type 'exceptions.NameError'> global name 'aa' is not defined

<type 'exceptions.UnboundLocalError'> local variable 'cc' referenced before assignment


SyntaxError: can not delete variable 'a' referenced in nested scope


- Each assignment or import statement occurs 
    - within a block defined by a class or function definition 
    - or at the module level (the top-level code block).
    
    
- If a name binding operation occurs anywhere within a code block, all uses of the name within the block are treated as **references** to the current block. 
    - This can lead to errors when a name is used within a block before it is bound. 
    - This rule is subtle: 
        - Python lacks declarations and allows name binding operations to occur anywhere within a code block. 
        - The local variables of a code block can be determined by scanning the entire text of the block for name binding operations.
        
        
- If the **global** statement occurs within a block, all uses of the name specified in the statement refer to the binding of that name in the **top-level** namespace. 
    - Names are resolved in the top-level namespace by searching the global namespace, i.e. 
        - the namespace of the module containing the code block, and 
        - the builtins namespace, the namespace of the module `__builtin__`. 
    - The global namespace is searched first. 
    - If the name is not found there, the builtins namespace is searched. 
    - The global statement must **precede** all uses of the name.
    
    
- The **builtins namespace** associated with the execution of a code block is actually found by looking up the name `__builtins__` in its global namespace; 
    - this should be a dictionary or a module (in the latter case the module’s dictionary is used). 
- By default, when in the `__main__` module, `__builtins__` is the built-in module `__builtin__` (note: **no ‘s’**); 
    - when in any other module, `__builtins__` is an alias for the dictionary of the `__builtin__` module itself. 
    - `__builtins__` can be set to a user-created dictionary to create a weak form of restricted execution.
    
    
- **CPython implementation detail**: 
    - Users should not touch `__builtins__`; it is strictly an implementation detail. 
    - Users wanting to override values in the builtins namespace should import the `__builtin__` (**no ‘s’**) module and modify its attributes appropriately.


In [2]:
print type(__builtins__), __builtins__
print type(__builtin__), __builtin__
print dir(__builtins__)[:5]
print dir(__builtin__)[:5]
print __builtins__ == __builtin__
import __builtin__ as bi
print __builtins__ == bi

<type 'module'> <module '__builtin__' (built-in)>
<type 'module'> <module '__builtin__' (built-in)>
['ArithmeticError', 'AssertionError', 'AttributeError', 'BaseException', 'BufferError']
['ArithmeticError', 'AssertionError', 'AttributeError', 'BaseException', 'BufferError']
True
True


- The namespace for a module is automatically created the **first time** a module is imported. 
- The main module for a script is always called `__main__`.


- The ***global*** statement has the same scope as a name binding operation in the same block. 
- If the nearest enclosing scope for a free variable contains a **global** statement, the **free** variable is treated as a **global**.


- A **class** definition is an executable statement that may use and define names. 
    - These references follow the normal rules for name resolution. 
    - The namespace of the class definition becomes the *attribute dictionary* of the class. 
    - Names defined at the class scope are **not** visible in methods.

In [11]:
class C:
    a = 33
    def method_a(self): 
        print a # NameError: global name 'a' is not defined
        
C().method_a()

NameError: global name 'a' is not defined

In [12]:
class C:
    a = 33
    def method_a(self): 
        print C.a # use C
        
C().method_a()

33


In [2]:
class C(object):
    a = 33
    def method_a(self): 
        print self.a # use self to refer it
        
C().method_a()

33


In [5]:
myvariable = 5
def func():
    myvariable = 6    # creates a new "local" variable.  
                      # Doesn't affect the global version
    print myvariable  # prints 6

func()
print myvariable   #prints 5

6
5


In [6]:
myvariable = 5
def func():
    global myvariable # defined at the global (module-level) scope
    myvariable = 6    # changes `myvariable` at the global scope
    print myvariable  # prints 6

func()
print myvariable  #prints 6 now because we were able 
                  #to modify the reference in the function

6
6


In [11]:
def f(): 
    print s 
s = "I hate spam"
f()

print 

def f(): 
    s = "Me too."
    print s 
s = "I hate spam." 
f()
print s

print

def f(): 
    print s        # UnboundLocalError: local variable 's' referenced before assignment
    s = "Me too."  # this is because Python "assumes" that we want a local variable due to the assignment to s
    print s
s = "I hate spam." 
try:
    f()
except Exception as e:
    print type(e), e
print s

print

def f():
    global s       # defined at the global (module-level) scope
    print s
    s = "Me too."
    print s 
s = "I hate spam!" 
f()
print s

print

def f():
    s2 = "I am globally not known"
    print s2 
f()
try:
    print s2           # local variable cannot be accessed outside
except Exception as e:
    print type(e), e   # <type 'exceptions.NameError'> name 's2' is not defined

print

# a deliberate combination of local and global variables 
# and function parameters
def foo(x, y):
    global a
    a = 42
    x,y = y,x
    b = 33
    b = 17
    c = 100
    print a,b,x,y      # 42 17 4 17
                       # a is changed globally
                       # b, c is local
                       # x,y swapped
a,b,x,y = 1,15,3,4
print a,b,x,y          # 1  15 3 4
foo(17,4)
print a,b,x,y          # 42 15 3 4
                       # a is changed globally
                       # b is the global one
                       # x,y are the global ones

I hate spam

Me too.
I hate spam.

<type 'exceptions.UnboundLocalError'> local variable 's' referenced before assignment
I hate spam.

I hate spam!
Me too.
Me too.

I am globally not known
<type 'exceptions.NameError'> name 's2' is not defined

1 15 3 4
42 17 4 17
42 15 3 4


In [3]:
""" PYTHON 3.x ONLY!
Below is an example of 

    nonlocal
    
statement. 
"""
def func1():
    x = 3
    def func2():
        print("x=",x,"func2")
        y = 4
        def func3():
            nonlocal x  #try it with nonlocal commented out as well.  See the difference.
            print("x=",x,"func3")
            print("y=",y,"func3")
            z = 5
            print("z=",z,"func3")
            x = 10

        func3()

    func2()
    print("x=",x,"func1")

func1()

def func1():
    x = 3
    def func2():
        print("x=",x,"func2")
        y = 4
        def func3():
            # nonlocal x          # try it with nonlocal commented out as well.  See the difference.
            print("x=",x,"func3") # UnboundLocalError: local variable 'x' referenced before assignment
            print("y=",y,"func3")
            z = 5
            print("z=",z,"func3")
            x = 10

        func3()

    func2()
    print("x=",x,"func1")

func1()

x= 3 func2
x= 3 func3
y= 4 func3
z= 5 func3
x= 10 func1
x= 3 func2


UnboundLocalError: local variable 'x' referenced before assignment

### 4.1.1. Interaction with dynamic features

- There are several cases where Python statements are **illegal** when used in conjunction with nested scopes that contain **free variables**.

    - If a variable is referenced in an enclosing scope, it is illegal to delete the name. 
    - An error will be reported at compile time.


In [18]:
def func1():
    
    a = 1
    print 'a in func1():', a
    
    def nested_func():
        print 'a referenced in nested_func():', a
    
    del a # SyntaxError: can not delete variable 'a' referenced in nested scope
    
func1() 

SyntaxError: can not delete variable 'a' referenced in nested scope

- If the wild card form of import — import * — is used in a function and the function contains or is a nested block with free variables, the compiler will raise a SyntaxError.


In [4]:
def f1():
    a = 1
    def foo():
        from os import * # SyntaxWarning, SyntaxError
        print a
    foo()
f1()
    

<ipython-input-4-b2b98a116fe3>:3: SyntaxWarning: import * only allowed at module level
  def foo():


SyntaxError: import * is not allowed in function 'foo' because it is a nested function (<ipython-input-4-b2b98a116fe3>, line 4)

In [5]:
def f1():
    def foo():
        from os import * # SyntaxWarning
        print 1
    foo()
f1()

1


<ipython-input-5-193c6728fa41>:2: SyntaxWarning: import * only allowed at module level
  def foo():



- If exec is used in a function and the function contains or is a nested block with free variables, the compiler will raise a SyntaxError unless the exec explicitly specifies the local namespace for the exec. 
- In other words, 
    - exec obj would be illegal, but 
    - exec obj in ns would be legal.


- The eval(), execfile(), and input() functions and the exec statement do not have access to the full environment for resolving names. 
    - Names may be resolved in the local and global namespaces of the caller. 
    - Free variables are not resolved in the nearest enclosing namespace, but in the global namespace. 
    - The exec statement and the eval() and execfile() functions have optional arguments to override the global and local namespace. 
        - If only one namespace is specified, it is used for both.

In [6]:
def f1():
    a = 1
    def foo():
        exec('print 66') # SyntaxWarning, SyntaxError
        print a
    foo()
f1()

SyntaxError: unqualified exec is not allowed in function 'foo' because it is a nested function (<ipython-input-6-e54ec2c4f1c2>, line 4)

In [8]:
def f1():
    def foo():
        exec('print 66') 
        print 1
    foo()
f1()

66
1


### 4.2. Exceptions

- Exceptions are a means of breaking out of the normal flow of control of a code block in order to handle errors or other exceptional conditions. 
    - An exception is raised at the point where the error is detected; 
    - it may be handled by the surrounding code block or by any code block that directly or indirectly invoked the code block where the error occurred.
    
    
- The Python interpreter raises an exception when it detects a run-time error (such as division by zero). 
- A Python program can also explicitly raise an exception with the raise statement. 
- Exception handlers are specified with the **try ... except** statement. 
- The **finally** clause of such a statement can be used to specify **cleanup** code which does not handle the exception, but is executed no matter an exception occurred or not in the preceding code.


- Python uses the “**termination**” model of error handling: 
    - an exception handler can find out what happened and continue execution at an outer level, 
    - but it cannot repair the cause of the error and retry the failing operation 
        - (except by re-entering the offending piece of code from the top).
        
        
- When an exception is not handled at all, the interpreter terminates execution of the program, or returns to its interactive main loop. 
    - In either case, it prints a stack backtrace, except when the exception is **SystemExit**.
    
    
- Exceptions are identified by class instances. 
- The except clause is selected depending on the class of the instance: 
    - it must reference the class of the instance or a base class thereof. 
    - The instance can be received by the handler and can carry additional information about the exceptional condition.
    
    
- Exceptions can also be identified by strings, 
    - in which case the except clause is selected by object identity. 
    - An arbitrary value can be raised along with the identifying string which can be passed to the handler.
- **Note**:
    - Messages to exceptions are not part of the Python API. 
    - Their contents may change from one version of Python to the next without warning and 
    - should not be relied on by code which will run under multiple versions of the interpreter.